#Config for Microsoft Azure for this notebook in Databricks

###Step 1 - Create Reource Group
Resource group -> New Resource Group -> name - RG_databricks_masterclass
->Create

###Step 2 - Create Storage
- Resource Groups
- Open your group
- Create
- Search 'Storage' - Select 'Storage Account' by Microsoft
- Pick correct Resoure Group
- Create unique storage account name - datalakemercifulsamurai
- Select redundancy - LRS (Locally Redundant Storage - Data is replicated within same data cluster - cheapest option)

###Step 3 - Create containers
Create 2 containers - 
- source - Upload csv file to source container
- destination

###Step 4 - Create Dtabricks Workspace
- Open Resource Group
- Create
- Databricks - Search and add the one from microsoft
- Give name and create using the following
- Open resource after creation and open workspace

###Step 5 - Create compute for Databricks
- Open Workspace
- Compute -> Create compute
- Single mode - Check
- Photon Acceleration - Uncheck

###Step 6 - Create notebook
- Create a workspace
- Create notebook
- Connect cluster to notebook

###Step 7 - Service Principle App
- Open Azure Portal
- Microsoft Entra ID
- Manage
- App Registrations
- Add
- Name your app
- Copy and keep - Application ID, Tenant ID
- In this App - Open Mange -> Certifications & Secrets
- Create new Secret Key and copy immediately

###Step 8 - Assign access to the datalake
- Open Resource group
- Open data-storage-account
- Containers
- IAM
- Add new role
- Search 'Storage blob data contributor - select it and click next
- +Select members
- Search principle app with name which was just created

###Step 8 - Create Key Vault for secrets
- Open Azure
- Open 'Keyvault' -> Create
- Select resource-group
- Name keyvault
- Access configuration(Next) - Select Vault Access policy
- Create 

###Step 9 - Role Assignment
- Open Key vault -> IAM
- +Add -> Add role assignment
- Search 'Key Vault Admin' -> Next
- Pick user (email)
- Select -> Review+Assign

###Step 10 - Add secrets
- Open keyvault
- Objects -> Secrets -> Generate/Import
- Put name and put key in value (Create 3 secrets for app-id, tenant-id, app-secret-key of the service principle app)

###Step 11 - Create databricks scope
- Open databricks workspace in Azure
- Copy the url
- Paste it in a new tab like ->[<url]#secrets/createScope
- Name - use a name (mercifulscope)
- DNS name - vault URL from Azure Key Vault Properties
- Resource ID - Resource ID from same Settings/Properties
- Create
- Now, we can use dbutils.secret with this scope to get and use the keys stored securely in the keyvault

###Step 12 - Establish Connection to Azure in notebook
- Open link https://learn.microsoft.com/en-us/azure/databricks/connect/storage/azure-storage
- Copy spark.conf.set template
- Open notebook and paste
- Put storage name
- Use dbutils.secrets.list(scope='scope-name') to list avaiable keys
- Use dbutils.secret.get(scope='scope-name',key='key-name') to get the app-id and app-secret stored in keyvault and use here

###Step 13 - Setup for External Tables
- Open Azure Portal
- Open Resource Group
- Create -> Access Connector for Azure Databricks
- Open Datalake
- Goto IAM
- Add - Storage Blob Data Contributor
- Assign Access to Managed Identity
- Select Members -> Managed Identity(Access Connector for Azure Databricks)
- Open Databricks Workspace
- Goto Catalog -> External Data (top-right of screen)
- Create External Location -> URL = abfss://[container]@[datalake-name].dfs.core.windows.net/[folder-path]
This location should be where the data for External Table is stored

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.secrets.list(scope='mercifulscope')

In [0]:
app_id = dbutils.secrets.get(scope='mercifulscope',key='app-id')
app_secret_key = dbutils.secrets.get(scope='mercifulscope',key='app-secret-key')

In [0]:
spark.conf.set("fs.azure.account.auth.type.datalakemercifulsamurai.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.datalakemercifulsamurai.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.datalakemercifulsamurai.dfs.core.windows.net", app_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.datalakemercifulsamurai.dfs.core.windows.net", app_secret_key)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.datalakemercifulsamurai.dfs.core.windows.net", "https://login.microsoftonline.com/3f20cb52-8282-4c01-9c13-59e5bea78878/oauth2/token")

In [0]:
dbutils.fs.ls("abfss://sparkdata@datalakemercifulsamurai.dfs.core.windows.net/")